In [1]:
import nltk
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.models import Sequential
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
train_data = pd.read_csv("../datasets/labled/train.csv")

In [3]:
train_data


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [4]:
train_data.dropna(inplace=True)

In [5]:
train_data


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [6]:
ps = PorterStemmer()

In [7]:
sentence_length = 5000
padding_length = 1000
dimension = 100

In [22]:
def pre_process_data(data):
    text = data['DocText']
    sentence_list = list(map(lambda x: "". join(ps.stem(word) for word in re.sub("[^A-Za-z]", " ", x.lower()) if word not in stopwords.words("english")), text.values))
    encoded_sentences = [one_hot(sentence, sentence_length) for sentence in sentence_list]
    padded_sentences = pad_sequences(encoded_sentences, maxlen=padding_length, padding='pre')
    return padded_sentences

In [9]:
sen_list = pre_process_data(train_data)

In [10]:
sen_list



array([[   0,    0,    0, ..., 3547, 2925, 1306],
       [   0,    0,    0, ..., 2816, 1408,  427],
       [   0,    0,    0, ...,   32,  160,  750],
       ...,
       [   0,    0,    0, ..., 1404, 4065, 3605],
       [   0,    0,    0, ..., 1362, 2816, 1588],
       [   0,    0,    0, ..., 4936,  854, 2897]])

In [11]:
model = Sequential()
model.add(Embedding(sentence_length, dimension, input_length=padding_length))
model.add(LSTM(500))
#model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', metrics='accuracy', optimizer='adam')

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 100)         500000    
                                                                 
 lstm (LSTM)                 (None, 500)               1202000   
                                                                 
 dense (Dense)               (None, 1)                 501       
                                                                 
Total params: 1,702,501
Trainable params: 1,702,501
Non-trainable params: 0
_________________________________________________________________


In [12]:
x_input = np.array(sen_list)
y_input = np.array(train_data['label'].values)
x_train, x_test, y_train, y_test = train_test_split(x_input, y_input, test_size=0.2, random_state=1)
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=2, batch_size=64)

Epoch 1/2
229/229 [==============================] - 2693s 12s/step - loss: 0.3149 - accuracy: 0.8527 - val_loss: 0.2049 - val_accuracy: 0.9155
Epoch 2/2
229/229 [==============================] - 2606s 11s/step - loss: 0.1650 - accuracy: 0.9316 - val_loss: 0.2021 - val_accuracy: 0.9202


In [28]:
test_data = pd.read_csv("../datasets/polifact/Politifact.test2_hard.tsv",sep='\t', header=0)
test_data.fillna(inplace=True, method='pad')
test_data.drop(inplace=True,columns=['QueryText', 'QueryImages', 'DocID', 'DocImages'])
test_data

,QueryID,DocText,Label
0,3,Sen. Marco Rubio said his parents 'came to Ame...,0
1,3,"No, 15,000 people did not vote for Harambe in ...",0
2,3,Trump wrong on Sen. Blumenthal's CNN interview...,0
3,3,Viral graphic says Japan keeps out radical Isl...,0
4,3,"No, the Obama administration didn't invent the...",0
...,...,...,...
7795,2004,Was Chelsea Clinton charged with fraud as Clin...,0
7796,2004,Trump: U.S. spends more than 'almost any other...,0
7797,2004,Fox News host wrong that no law forbids Russia...,0
7798,2004,Trump wrongly says $6 billion went missing at ...,0


In [ ]:
test_sentence_list = pre_process_data(test_data)


In [17]:
test_input = np.array(test_sentence_list)
test_output = model.predict(test_input)
submission_df = pd.DataFrame({'id': test_data['QueryID'],'title' : test_data['DocText'], 'label': (test_output.flatten().reshape(-1,) > 0.5).astype('int')})

In [18]:
submission_df.to_csv("submissionhard.csv", index=False)